In [40]:
import numpy as np

In [41]:
from nn_labs.nn.layers import BaseLayer
from nn_labs.nn.activation_functions import BaseActivationFunction

from numpy.typing import NDArray

class Network:
    def __init__(self):
        self.layers: list[BaseLayer | BaseActivationFunction] = []
    
    def add_layer(self, l: BaseLayer | BaseActivationFunction):
        self.layers.append(l)
        
    def forward(self, x_in: NDArray) -> NDArray:
        current = x_in
        
        for layer in self.layers:
            current = layer.forward(current)
            
        return current
    
    def backward(self, d_loss: NDArray):
        current_gradient = d_loss
        
        for layer in reversed(self.layers):
            layer.backward(current_gradient)
            
            current_gradient = layer.d_inputs

In [42]:
from nn_labs.nn.layers import Conv2D, DenseLayer, Flatten, MaxPool2D
from nn_labs.nn.activation_functions import ReLU, Sigmoid

bob_net = Network()

bob_net.add_layer(Conv2D(1, 2, 16, 1))
bob_net.add_layer(MaxPool2D(2, 2))
bob_net.add_layer(Conv2D(16, 2, 16, 1))
bob_net.add_layer(MaxPool2D(2, 2))
bob_net.add_layer(Flatten())
bob_net.add_layer(DenseLayer(256, 400))
bob_net.add_layer(ReLU())
bob_net.add_layer(DenseLayer(400, 784))
bob_net.add_layer(Sigmoid())

In [43]:
x = np.ones((100, 1, 28, 28))

out = bob_net.forward(x)

out.shape

(100, 784)

In [44]:
from nn_labs.nn.perf import MSELoss
from nn_labs.image_net.load import load_fashion_mnist

(X_train, Y_train), (X_test, Y_test) = load_fashion_mnist()

X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

EPOCHS = 2000
BATCH_SIZE = 4086
LEARNING_RATE = 10e-3
batch_count = X_train.shape[0] // BATCH_SIZE

loss_fn = MSELoss()

for epoch in range(EPOCHS):
    print(f"Epoch: {epoch}")
    for batch_no in range(batch_count):
        start_idx = batch_no * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE

        x = X_train[start_idx:end_idx]
        y = Y_train[start_idx:end_idx]
        
        x_cmp = x.reshape(BATCH_SIZE, 784)
        
        prediction = bob_net.forward(x)
        loss = loss_fn.calculate(prediction, x_cmp)
        
        loss_fn.backward(prediction, x_cmp)
        bob_net.backward(loss_fn.d_inputs)


Epoch: 0
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/philip/code/uni/nn/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/xv/v7b525y50112w8rdmkdf28lr0000gn/T/ipykernel_52218/1345848905.py", line 31, in <module>
    bob_net.backward(loss_fn.d_inputs)
  File "/var/folders/xv/v7b525y50112w8rdmkdf28lr0000gn/T/ipykernel_52218/2344374159.py", line 25, in backward
    layer.backward(current_gradient)
  File "/Users/philip/code/uni/nn/nn_labs/nn/layers.py", line 31, in backward
NotImplementedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/philip/code/uni/nn/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/philip/code/uni/nn/.venv/lib/python3